In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
newsgroups_train=fetch_20newsgroups(subset='train',shuffle=True)
newsgroups_test=fetch_20newsgroups(subset='test',shuffle=True)

In [2]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
newsgroups_train.DESCR

'.. _20newsgroups_dataset:\n\nThe 20 newsgroups text dataset\n------------------------------\n\nThe 20 newsgroups dataset comprises around 18000 newsgroups posts on\n20 topics split in two subsets: one for training (or development)\nand the other one for testing (or for performance evaluation). The split\nbetween the train and test set is based upon a messages posted before\nand after a specific date.\n\nThis module contains two loaders. The first one,\n:func:`sklearn.datasets.fetch_20newsgroups`,\nreturns a list of the raw texts that can be fed to text feature\nextractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`\nwith custom parameters so as to extract feature vectors.\nThe second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,\nreturns ready-to-use features, i.e., it is not necessary to use a feature\nextractor.\n\n**Data Set Characteristics:**\n\n    =================   ==========\n    Classes                     20\n    Samples total           

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [5]:

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
print(WordNetLemmatizer().lemmatize('went',pos='v'))

go


In [9]:
import pandas as pd
stemmer=SnowballStemmer('english')
original_words=['caresses','flies','dies','mules','denied','died','agreed','owned','humbled','sized','meeting','stating','siezing','itemization','sensationa','traditional','reference','colonizer','plotted']
singles=[stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'Original word':original_words,'stemmed':singles})


,Original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3:
            result.append(lemmatize_stemming(token))
 
    return result


In [12]:
document=50
doc_sample='This disk had faild many times.I would like to get it replaced'
print('Original Document: ')
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n Tokenized and lemmatized document: ')
print(preprocess(doc_sample))

Original Document: 
['This', 'disk', 'had', 'faild', 'many', 'times.I', 'would', 'like', 'to', 'get', 'it', 'replaced']


 Tokenized and lemmatized document: 
['disk', 'faild', 'time', 'like', 'replac']


In [13]:
processed_docs=[]
for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))
print(processed_docs[:2])

[['lerxst', 'thing', 'subject', 'nntp', 'post', 'host', 'organ', 'univers', 'maryland', 'colleg', 'park', 'line', 'wonder', 'enlighten', 'door', 'sport', 'look', 'late', 'earli', 'call', 'bricklin', 'door', 'small', 'addit', 'bumper', 'separ', 'rest', 'bodi', 'know', 'tellm', 'model', 'engin', 'spec', 'year', 'product', 'histori', 'info', 'funki', 'look', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst'], ['guykuo', 'carson', 'washington', 'subject', 'clock', 'poll', 'final', 'summari', 'final', 'clock', 'report', 'keyword', 'acceler', 'clock', 'upgrad', 'articl', 'shelley', 'qvfo', 'innc', 'organ', 'univers', 'washington', 'line', 'nntp', 'post', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'soul', 'upgrad', 'clock', 'oscil', 'share', 'experi', 'poll', 'send', 'brief', 'messag', 'detail', 'experi', 'procedur', 'speed', 'attain', 'rat', 'speed', 'card', 'adapt', 'heat', 'sink', 'hour', 'usag', 'floppi', 'disk', 'function', 'floppi', 'especi', 'request', 'summar', 'day',

In [14]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [15]:
count =0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count >10:
        break

0 addit
1 bodi
2 bricklin
3 bring
4 bumper
5 call
6 colleg
7 door
8 earli
9 engin
10 enlighten


In [16]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [17]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
document_num=20
bow_doc_x=bow_corpus[document_num]
for i in range(len(bow_doc_x)):
    print('Word {} (\"{}\") appears {} time:'.format(bow_doc_x[i][0],dictionary[bow_doc_x[i][0]],bow_doc_x[i][1]))


Word 18 ("rest") appears 1 time:
Word 166 ("clear") appears 1 time:
Word 336 ("refer") appears 1 time:
Word 350 ("true") appears 1 time:
Word 391 ("technolog") appears 1 time:
Word 437 ("christian") appears 1 time:
Word 453 ("exampl") appears 1 time:
Word 476 ("jew") appears 1 time:
Word 480 ("lead") appears 1 time:
Word 482 ("littl") appears 3 time:
Word 520 ("wors") appears 2 time:
Word 721 ("keith") appears 3 time:
Word 732 ("punish") appears 1 time:
Word 803 ("california") appears 1 time:
Word 859 ("institut") appears 1 time:
Word 917 ("similar") appears 1 time:
Word 990 ("allan") appears 1 time:
Word 991 ("anti") appears 1 time:
Word 992 ("arriv") appears 1 time:
Word 993 ("austria") appears 1 time:
Word 994 ("caltech") appears 2 time:
Word 995 ("distinguish") appears 1 time:
Word 996 ("german") appears 1 time:
Word 997 ("germani") appears 3 time:
Word 998 ("hitler") appears 1 time:
Word 999 ("livesey") appears 2 time:
Word 1000 ("motto") appears 2 time:
Word 1001 ("order") appear

In [18]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]
document_num=45
bow_doc_x=bow_corpus[document_num]
for i in range(len(bow_doc_x)):
    print('Word {} (\"{}\") appears {} time:'.format(bow_doc_x[i][0],dictionary[bow_doc_x[i][0]],bow_doc_x[i][1]))


Word 8 ("engin") appears 1 time:
Word 82 ("electr") appears 1 time:
Word 109 ("probabl") appears 1 time:
Word 150 ("robert") appears 1 time:
Word 165 ("check") appears 1 time:
Word 213 ("cours") appears 2 time:
Word 388 ("servic") appears 1 time:
Word 398 ("wrong") appears 1 time:
Word 1024 ("pittsburgh") appears 1 time:
Word 1106 ("andrew") appears 2 time:
Word 1496 ("american") appears 1 time:
Word 1654 ("auvm") appears 1 time:
Word 1655 ("carnegi") appears 1 time:
Word 1656 ("charg") appears 1 time:
Word 1657 ("freshman") appears 1 time:
Word 1658 ("honesti") appears 1 time:
Word 1659 ("mellon") appears 1 time:


# Build a model - Latent Dirichlet Allocation (LDA)

In [19]:
lda_model=gensim.models.LdaMulticore(bow_corpus,num_topics=8,id2word=dictionary,passes=10)

In [20]:
for idx,topic in lda_model.print_topics(-1):
    print('Topic: {} \n Words: {}'.format(idx,topic))
    print('\n')


Topic: 0 
 Words: 0.006*"bike" + 0.005*"team" + 0.005*"game" + 0.003*"homosexu" + 0.003*"run" + 0.003*"play" + 0.003*"player" + 0.003*"pitch" + 0.003*"season" + 0.003*"defens"


Topic: 1 
 Words: 0.008*"armenian" + 0.007*"govern" + 0.005*"israel" + 0.005*"kill" + 0.005*"turkish" + 0.005*"isra" + 0.004*"live" + 0.003*"weapon" + 0.003*"jew" + 0.003*"american"


Topic: 2 
 Words: 0.011*"game" + 0.009*"team" + 0.008*"play" + 0.006*"player" + 0.006*"hockey" + 0.004*"entri" + 0.004*"season" + 0.003*"canada" + 0.003*"score" + 0.003*"toronto"


Topic: 3 
 Words: 0.009*"window" + 0.007*"drive" + 0.006*"card" + 0.006*"driver" + 0.006*"sale" + 0.005*"speed" + 0.004*"price" + 0.004*"appl" + 0.004*"monitor" + 0.004*"sell"


Topic: 4 
 Words: 0.011*"file" + 0.008*"window" + 0.008*"program" + 0.005*"data" + 0.005*"avail" + 0.005*"imag" + 0.005*"version" + 0.004*"graphic" + 0.004*"chip" + 0.004*"space"


Topic: 5 
 Words: 0.007*"scienc" + 0.005*"pitt" + 0.004*"space" + 0.003*"bank" + 0.003*"gordon" + 